In [1]:
# !pip install unsloth # install unsloth
# !pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!


In [2]:
!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 1.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 7.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 78.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.2 MB/s eta 0:00:00:0

In [3]:
# Step3: Import necessary libraries
import unsloth

from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-07 16:20:21.311062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754583621.505527      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754583621.559603      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
import os
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"


In [5]:
# # Step4: Check HF token
# from google.colab import userdata
# hf_token = userdata.get('HF_TOKEN')
# login(hf_token)

from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
import wandb

user_secrets = UserSecretsClient()

# Get Hugging Face token
hf_token = user_secrets.get_secret("HF_TOKEN")
login(hf_token)

# # Get WandB token
# wandb_token = user_secrets.get_secret("WANDB_API_TOKEN")
# wandb.login(key=wandb_token)


In [6]:
# Optional: Check GPU availability
# Test if CUDA is available
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU device: Tesla P100-PCIE-16GB


In [7]:
# Step5: Setup pretrained DeepSeek-R1
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
max_sequence_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.8.2: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [8]:

# Step6: Setup system prompt
prompt_style = """
Below is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.

Before crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.

### Task:
You are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.

### Query:
{}

### Answer:
{}
"""

In [9]:

# Step7: Run Inference on the model

# Define a test question
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or
              sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""

FastLanguageModel.for_inference(model)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)


print(response)

['<｜begin▁of▁sentence｜>\nBelow is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.\n\nBefore crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.\n\n### Task:\nYou are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.\n\n### Query:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or\n              sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,\n              what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Answer:\n\nCystometry, also known as urodynamic testing, is a diagnostic procedure used

In [10]:
print(response[0].split("### Answer:")[1])



Cystometry, also known as urodynamic testing, is a diagnostic procedure used to evaluate the function of the lower urinary tract. It involves filling the bladder and measuring how much urine is left in the bladder (residual volume) and how the bladder contracts (detrusor contractions) during the filling process. 

In this case, the patient is a 61-year-old woman with a history of involuntary urine loss, particularly during activities like coughing or sneezing. These are common triggers for stress urinary incontinence (SUI), which is often due to weak pelvic floor muscles or urethral issues. She has no leakage at night, which suggests that her bladder capacity and nighttime voiding are not a primary concern.

Upon undergoing a gynecological exam and Q-tip test, the findings likely indicate that she does not have urethral obstruction or significant issues with the urethra that could contribute to her symptoms. The Q-tip test, which involves inserting a cotton-tipped probe into the uret

In [11]:
# Step8: Setup fine-tuning

# Load Dataset
medical_dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split = "train[:500]", trust_remote_code = True)


README.md: 0.00B [00:00, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [12]:
EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which tells the model when to stop generating text during training
EOS_TOKEN


'<｜end▁of▁sentence｜>'

In [13]:
### Finetuning
# Updated training prompt style to add  tag
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:

{}

{}"""


In [14]:
# Prepare the data for fine-tuning

def preprocess_input_data(examples):
  inputs = examples["Question"]
  cots = examples["Complex_CoT"]
  outputs = examples["Response"]

  texts = []

  for input, cot, output in zip(inputs, cots, outputs):
    text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
    texts.append(text)

  return {
      "texts" : texts,
  }

In [15]:
finetune_dataset = medical_dataset.map(preprocess_input_data, batched = True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [16]:
finetune_dataset

Dataset({
    features: ['Question', 'Complex_CoT', 'Response', 'texts'],
    num_rows: 500
})

In [17]:
# Step9: Setup/Apply LoRA finetuning to the model

model_lora = FastLanguageModel.get_peft_model(
    model = model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3047,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2025.8.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [18]:
# Add this before creating the trainer
if hasattr(model, '_unwrapped_old_generate'):
    del model._unwrapped_old_generate

In [19]:
# def formatting_func(example):
#     # Assuming your dataset has 'instruction' and 'response' fields
#     text = f"### Instruction:\n{example['instruction']}\n### Response:\n{example['response']}"
#     return [text]

def formatting_func(examples):
  inputs = examples["Question"]
  cots = examples["Complex_CoT"]
  outputs = examples["Response"]

  texts = []

  for input, cot, output in zip(inputs, cots, outputs):
    text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
    texts.append(text)

  return texts

In [20]:
trainer = SFTTrainer(
    model = model_lora,
    tokenizer = tokenizer,
    train_dataset = finetune_dataset,
    formatting_func=formatting_func,
    dataset_text_field = "texts",
    max_seq_length = max_sequence_length,
    dataset_num_proc = 2,

    # Define training args
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/500 [00:00<?, ? examples/s]

In [21]:
# Setup WANDB
wandb_token = user_secrets.get_secret("WANDB_API_TOKEN")
# login to wandb
wandb.login(key=wandb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-on-Medical-CoT-Dataset',
    job_type="training",
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nasratjahan166 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [22]:
import os
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"


In [23]:
# Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.944400
20,1.444900
30,1.421300
40,1.376500
50,1.401500
60,1.380400


In [26]:

wandb.finish()

In [27]:
# Step10: Testing after fine-tuning
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing
              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response)

["<｜begin▁of▁sentence｜>\nBelow is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.\n\nBefore crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.\n\n### Task:\nYou are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.\n\n### Query:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing\n              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,\n              what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Answer:\n\nAlright, let's think about this. This woman is 61 and has been dealing with 

In [28]:

print(response[0].split("### Answer:")[1])



Alright, let's think about this. This woman is 61 and has been dealing with involuntary urine loss for a long time, especially when she coughs or sneezes. That's a classic sign of urgency. She hasn't had any leakage at night, though. That's interesting because it might mean her bladder can hold up when she's asleep.

Okay, let's move on to the exam. She went for a pelvic exam and they did a Q-tip test. Now, the Q-tip test is a quick way to check how much urine she can hold before leaking. If the test shows she can hold a good amount, it suggests her bladder capacity is decent. This makes me think her residual volume isn't too low.

Now, let's consider what cystometry would show. It's a detailed test that looks at how much urine she can hold and how her bladder behaves. If her residual volume is high, that means her bladder isn't emptying completely when she urinates. That could explain why she's having urgency so often.

Oh, and what about the detrusor contractions? They're like the 

In [29]:
question = """A 59-year-old man presents with a fever, chills, night sweats, and generalized fatigue,
              and is found to have a 12 mm vegetation on the aortic valve. Blood cultures indicate gram-positive, catalase-negative,
              gamma-hemolytic cocci in chains that do not grow in a 6.5% NaCl medium.
              What is the most likely predisposing factor for this patient's condition?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response[0].split("### Answer:")[1])



Okay, so we have this 59-year-old guy who is really unwell. He's got a fever, chills, night sweats, and he's feeling really fatigued. That's a pretty big clue about how he's feeling. Then, when they check his heart, they find this vegetation on the aortic valve. That's definitely worrying. Now, the blood cultures show something interesting: gram-positive, catalase-negative, gamma-hemolytic cocci in chains that don't grow in a 6.5% NaCl medium. Hmm, that's a bit tricky. Let's unpack this.

First off, the gram-positive, catalase-negative cocci that are gamma-hemolytic and form chains are telling us we're dealing with Streptococcus species, probably Streptococcus viridans. Now, why would he have a vegetation on his aortic valve? That's usually something that happens when you've got an infection in the bloodstream, like sepsis, which can spread to the heart valves. So this is looking like an infection, likely endocarditis.

Now, why would he be predisposed to this? Let's think about his 